In [62]:
#importing important packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-for-ensemble-models/

In [110]:
pip install xgboost


     |████████████████████████████████| 1.8 MB 686 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [115]:
pip install lightgbm

     |████████████████████████████████| 1.3 MB 325 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [91]:
df=pd.read_csv('train_u6lujuX_CVtuZ9i.csv')

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [93]:

#filling missing values
df['Gender'].fillna('Male', inplace=True)

In [94]:
df['Loan_Status']=df['Loan_Status'].replace('Y',1)
df['Loan_Status']=df['Loan_Status'].replace('N',0)
#df['Dependents'] = df['Dependents'].astype(int)
df['Dependents'] = pd.to_numeric(df['Dependents'], errors='coerce')


In [95]:
df=df.drop('Loan_ID',axis=1)

In [96]:
cat=['Gender','Married','Education','Self_Employed','Property_Area']

In [97]:
# Create a list of the categorical column names
cat_cols = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']

# Create dummy variables for each categorical column and concatenate the result with the original DataFrame
df = pd.concat([df, pd.get_dummies(df[cat_cols], prefix=cat_cols)], axis=1)

# Drop the original categorical columns from the DataFrame
df.drop(cat_cols, axis=1, inplace=True)

In [98]:
df.isnull().sum()

Dependents                 66
ApplicantIncome             0
CoapplicantIncome           0
LoanAmount                 22
Loan_Amount_Term           14
Credit_History             50
Loan_Status                 0
Gender_Female               0
Gender_Male                 0
Married_No                  0
Married_Yes                 0
Education_Graduate          0
Education_Not Graduate      0
Self_Employed_No            0
Self_Employed_Yes           0
Property_Area_Rural         0
Property_Area_Semiurban     0
Property_Area_Urban         0
dtype: int64

In [99]:
df.dropna(inplace=True)

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472 entries, 1 to 613
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Dependents               472 non-null    float64
 1   ApplicantIncome          472 non-null    int64  
 2   CoapplicantIncome        472 non-null    float64
 3   LoanAmount               472 non-null    float64
 4   Loan_Amount_Term         472 non-null    float64
 5   Credit_History           472 non-null    float64
 6   Loan_Status              472 non-null    int64  
 7   Gender_Female            472 non-null    uint8  
 8   Gender_Male              472 non-null    uint8  
 9   Married_No               472 non-null    uint8  
 10  Married_Yes              472 non-null    uint8  
 11  Education_Graduate       472 non-null    uint8  
 12  Education_Not Graduate   472 non-null    uint8  
 13  Self_Employed_No         472 non-null    uint8  
 14  Self_Employed_Yes        4

In [73]:
df.isnull().sum()

Dependents                 0
ApplicantIncome            0
CoapplicantIncome          0
LoanAmount                 0
Loan_Amount_Term           0
Credit_History             0
Loan_Status                0
Gender_Female              0
Gender_Male                0
Married_No                 0
Married_Yes                0
Education_Graduate         0
Education_Not Graduate     0
Self_Employed_No           0
Self_Employed_Yes          0
Property_Area_Rural        0
Property_Area_Semiurban    0
Property_Area_Urban        0
dtype: int64

In [101]:
#split dataset into train and test

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, random_state=0)

x_train=train.drop('Loan_Status',axis=1)
col=x_train.columns
y_train=train['Loan_Status']

x_test=test.drop('Loan_Status',axis=1)
y_test=test['Loan_Status']

#create dummies
x_train=pd.get_dummies(x_train)
x_test=pd.get_dummies(x_test)
col_test=x_test.columns

In [102]:
scaler = StandardScaler()

# Fit the scaler to the DataFrame and transform the values
x_train = scaler.fit_transform(x_train)

# Convert the scaled values back to a DataFrame
x_train = pd.DataFrame(x_train, columns=col)

# Fit the scaler to the DataFrame and transform the values
x_test = scaler.fit_transform(x_test)

# Convert the scaled values back to a DataFrame
x_test = pd.DataFrame(x_test, columns=col_test)



# Bagging meta-estimator

In [103]:
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
model = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
model.fit(x_train, y_train)
model.score(x_test,y_test)


0.7746478873239436

In [90]:
y_train

427    0.662589
368    0.662589
444    0.662589
451    0.662589
347    0.662589
         ...   
323    0.662589
192   -1.509231
117    0.662589
47     0.662589
172   -1.509231
Name: Loan_Status, Length: 330, dtype: float64

In [104]:
from sklearn.ensemble import BaggingRegressor
model = BaggingRegressor(tree.DecisionTreeRegressor(random_state=1))
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.20883390607101937

# AdaBoost

In [105]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)


0.795774647887324

In [106]:
 from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor()
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.10051650539906232

# Gradient Boosting (GBM)

In [107]:
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier(learning_rate=0.01,random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.8098591549295775

In [108]:
from sklearn.ensemble import GradientBoostingRegressor
model= GradientBoostingRegressor()
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.14202577998178123

# XGBoost

In [111]:
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.7605633802816901

In [112]:
import xgboost as xgb
model=xgb.XGBRegressor()
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.07455549972087872

# Light GBM

In [117]:
import lightgbm as lgb
train_data=lgb.Dataset(x_train,label=y_train)
#define parameters
params = {'learning_rate':0.001}
model= lgb.train(params, train_data, 100) 
y_pred=model.predict(x_test)
for i in range(0,185):
   if y_pred[i]>=0.5: 
    y_pred[i]=1
else:
    y_pred[i]=0

OSError: dlopen(/Users/maya.bunkar/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/maya.bunkar/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)

# CatBoost

In [119]:
pip install catboost


     |████████████████████████████████| 22.0 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 5.1 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [120]:
from catboost import CatBoostClassifier
model=CatBoostClassifier()
categorical_features_indices = np.where(df.dtypes != np.float)[0]
model.fit(x_train,y_train,cat_features=([ 0,  1, 2, 3, 4, 10]),eval_set=(x_test, y_test))
model.score(x_test,y_test)

/var/folders/mg/1nb0m0gn4qj8h4g462q15w8m0000gn/T/ipykernel_71472/2043549330.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features_indices = np.where(df.dtypes != np.float)[0]


CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=0]=-0.7267191963205961 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [121]:
from catboost import CatBoostRegressor
model=CatBoostRegressor()
categorical_features_indices = np.where(df.dtypes != np.float)[0]
model.fit(x_train,y_train,cat_features=([ 0,  1, 2, 3, 4, 10]),eval_set=(x_test, y_test))
model.score(x_test,y_test)

/var/folders/mg/1nb0m0gn4qj8h4g462q15w8m0000gn/T/ipykernel_71472/2766753166.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features_indices = np.where(df.dtypes != np.float)[0]


CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=0]=-0.7267191963205961 : cat_features must be integer or string, real number values and NaN values should be converted to string.